TODO Luki format??

Co prioritizovat? https://github.com/zdenkas/human-tafazzin-variants-database/blob/main/TODO.txt
Konzistence cisel (pozic) a nukleotidu
Jake k tomu pouzit data?
Kolik nekonzistenci tam najdeme? Pak to poslat BSF


In [1]:
import pandas as pd
from Bio import SeqIO

In [2]:
import helpers

In [3]:
database_folder = '../database_versions'

In [4]:
input_path_prefix = '../database_versions/0001_2023-10-30-21-51-49-628455_Human-TAFAZZIN-Variants-Database_' 
# TODO get this automatically in next notebooks

output_path_prefix = helpers.create_database_output_path_prefix()

version number: 0003
database output path prefix: ../database_versions/0003_2024-01-11-18-52-40-068734_Human-TAFAZZIN-Variants-Database_


In [5]:
taz_cdna_path = '../data_external/Homo_sapiens_ENST00000601016_6_sequence.fa'
taz_positions_path = '../data_external/ExonsSpreadsheet-Homo_sapiens_Transcript_Exons_ENST00000601016_whole_table.csv'

### Load positions

in hg38

In [6]:
taz_positions = pd.read_csv(taz_positions_path)

In [7]:
taz_positions['Start'] = pd.to_numeric(taz_positions['Start'].str.replace(',', ''), errors='coerce')
taz_positions['Start'] = taz_positions['Start'].fillna(0).astype(int)

In [8]:
taz_positions['End'] = pd.to_numeric(taz_positions['End'].str.replace(',', ''), errors='coerce')
taz_positions['End'] = taz_positions['End'].fillna(0).astype(int)

In [9]:
taz_positions.head()

,No.,Exon / Intron,Start,End,Start Phase,End Phase,Length,Sequence
0,NaN,5' upstream sequence,0,0,NaN,NaN,NaN,"{""provisional"":{..."
1,1,ENSE00003022330,154411539,154411952,-,1,414,"{""url"":""/Homo_sa..."
2,,Intron 1-2,154411953,154412085,NaN,NaN,133,"{""provisional"":{..."
3,2,ENSE00003127263,154412086,154412214,1,1,129,"{""provisional"":{..."
4,,Intron 2-3,154412215,154413206,NaN,NaN,992,"{""provisional"":{..."


### Load cdna

In [10]:
for seq_record in SeqIO.parse(taz_cdna_path, "fasta"):
    #print(seq_record.id)
    #print(seq_record.name)
    print(seq_record.description)
    #print(repr(seq_record.seq))
    #print(len(seq_record))
    print('-----')

ENST00000601016.6 TAFAZZIN-213 cdna:protein_coding
-----
TAFAZZIN-213 cds:protein_coding
-----
TAFAZZIN-213 ENSE00003022330 exon:protein_coding
-----
TAFAZZIN-213 ENSE00003127263 exon:protein_coding
-----
TAFAZZIN-213 ENSE00003736420 exon:protein_coding
-----
TAFAZZIN-213 ENSE00003017673 exon:protein_coding
-----
TAFAZZIN-213 ENSE00003050674 exon:protein_coding
-----
TAFAZZIN-213 ENSE00003154518 exon:protein_coding
-----
TAFAZZIN-213 ENSE00003158525 exon:protein_coding
-----
TAFAZZIN-213 ENSE00003787276 exon:protein_coding
-----
TAFAZZIN-213 ENSE00003724812 exon:protein_coding
-----
TAFAZZIN-213 ENSE00003736683 exon:protein_coding
-----
TAFAZZIN-213 ENSE00003627163 exon:protein_coding
-----
TAFAZZIN-213 intron 1:protein_coding
-----
TAFAZZIN-213 intron 2:protein_coding
-----
TAFAZZIN-213 intron 3:protein_coding
-----
TAFAZZIN-213 intron 4:protein_coding
-----
TAFAZZIN-213 intron 5:protein_coding
-----
TAFAZZIN-213 intron 6:protein_coding
-----
TAFAZZIN-213 intron 7:protein_coding
-----

In [11]:
len(seq_record)

10188

In [12]:
seq_record.seq

Seq('GCTTTCCGGCGGTTGCACCGGGCCGGGGTGCCAGCGCCCGCCTTCCCGTTTCCT...AAA')

In [13]:
seq_record.seq[-1]

'A'

In [14]:
seq_record

SeqRecord(seq=Seq('GCTTTCCGGCGGTTGCACCGGGCCGGGGTGCCAGCGCCCGCCTTCCCGTTTCCT...AAA'), id='X', name='X', description='X dna:chromosome chromosome:GRCh38:X:154411539:154421726:1', dbxrefs=[])

### Load Tafazzin database

In [15]:
df_pathogenic = pd.read_csv(input_path_prefix + 'pathogenic.csv')
df_vus = pd.read_csv(input_path_prefix + 'vus.csv')
df_exon5 = pd.read_csv(input_path_prefix + 'exon5.csv')
df_benign = pd.read_csv(input_path_prefix + 'benign.csv')

In [16]:
df_pathogenic.head(1)

,Location,Location in Genome release 37 (hg19),Location in Genome release 38 (hg38),Protein Variant Type,Impact of Variant,DNA Modifications,Protein or mRNA Variants,Functional outcome (MLCL/CL ratio),Taffazin Functional motifs,Method of Validation,References,Source,Additional variants in other genes,Location and Order of Discovery,Notes,Unnamed: 15
0,Exon 1,X:153640189,X:154411852,Frameshift,NaN,c.9_10dupG,p.His4Alafs*130,MLCL/CL elevated,NaN,NaN,Ref. 1 (Pat.1); Ref. 80; Ref. 113,NaN,NaN,1-1,NaN,NaN


In [17]:
loc_38_col = 'Location in Genome release 38 (hg38)'
loc_37_col = 'Location in Genome release 37 (hg19)'
dna_modif_col = 'DNA Modifications'

In [18]:
def get_exon_intron_from_taz_positions(taz_database_38_loc, taz_positions=taz_positions):
    matching_rows = taz_positions[(taz_positions['Start'] <= taz_database_38_loc) & (taz_positions['End'] >= taz_database_38_loc)]
    #rows = #matching_rows['Exon / Intron'].values
    # If you expect only one match and want to get a single value
    if len(matching_rows) == 1:
        return matching_rows.iloc[0]
    else:
        print("Multiple or no matching rows found!")
        return None

In [19]:
def find_seq_record(search_string, taz_cdna_path=taz_cdna_path):
    # Use a generator expression to find the first matching seq_record
    seq_record = next((record for record in SeqIO.parse(taz_cdna_path, "fasta") if search_string in record.description), None)
    return seq_record


In [23]:
for key, row in df_pathogenic.iterrows():
    try:
        taz_database_38_loc = int(row[loc_38_col].split(':')[1])
        taz_database_37_loc = int(row[loc_37_col].split(':')[1])
        taz_database_dna_modif = row[dna_modif_col]
        row_from_taz_positions = get_exon_intron_from_taz_positions(taz_database_38_loc)
        exon_intron_from_taz_positions = row_from_taz_positions['Exon / Intron']
        start_from_taz_positions = row_from_taz_positions['Start']
        end_from_taz_positions = row_from_taz_positions['End']


        seq_record = find_seq_record(exon_intron_from_taz_positions)

        if seq_record:
            #print(f"Found record: {seq_record.description}")
            sequence = seq_record.seq
            pos_in_sequence = taz_database_38_loc - start_from_taz_positions
            expected_letter = sequence[pos_in_sequence]
            print(taz_database_dna_modif, expected_letter)
            assert taz_database_dna_modif.endswith(expected_letter)
            # TODO hg37, corner cases??
        else:
            print("No matching record found.")
        
    except Exception as ex:
        pass
        # TODO!! solve these !!print(key, row[loc_38_col]) #, ex)
    #break
        

c.9_10dupG G
Multiple or no matching rows found!
c.51G>A      G
c.51G>A     G
c.53_54delCC C
c.82_84del GTG G
c.83T>A        T
c.86G>A       G
No matching record found.
No matching record found.
No matching record found.
No matching record found.
No matching record found.
No matching record found.
No matching record found.
No matching record found.
No matching record found.
No matching record found.
No matching record found.
No matching record found.
c.118A>G    A
c.124delC    CTGA >TGA C
c.124delC    CTGA >TGA C
c.127A>C   A
c.129del C
Multiple or no matching rows found!
c.137A>T A
Multiple or no matching rows found!
c.143delGG A
c.145delG G
c.149T>C    T
c.149T>C     T
c.149T>C    T
c.149T>C    T
c.153C>G   C
c.153C>G    C
c.153C>G    C
c.154G > T G
c.157dupC    CTC > CCTC C
Multiple or no matching rows found!
c.161T>A   T
c.162C>G C
c.163G>T  G
Multiple or no matching rows found!
c.170G>T    G
c.170G>T  G
c.171delA A
c.184C>T C
c.185C>T   C
c.207C>G    C
c.207C>G     C
c.207C>G    C

In [ ]:
expected_letter

In [ ]:
taz_database_dna_modif

In [ ]:
sequence

In [ ]:
sequence[start_from_taz_positions - start_from_taz_positions]

In [ ]:
row_from_taz_positions

In [ ]:
exon_intron_from_taz_positions

In [ ]:
seq_record.seq

In [ ]:
matching_record.seq

In [ ]:
taz_positions

In [ ]:
taz_positions.Start

In [ ]:
taz_positions.head(20)

In [ ]:
taz_database_38

In [ ]:
taz_database_37

In [ ]:
taz_database_dna_modif

In [ ]:
df_pathogenic.Location.value_counts()